In [ ]:
%pip install mtcnn keras_facenet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


from mtcnn import MTCNN
from keras_facenet import FaceNet
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix

In [8]:
SAVE_MAIN_PATH = '/content/drive/MyDrive/'
MODEL_PATH = '/content/drive/MyDrive/model.h5'

valid_labels = ['Emilia', 'Joanna', 'Kacper', 'Karolina', 'Mirek', 'Sylwia', 'Łukasz', 'Mariusz', 'Ania']

In [9]:
def load_embeddings(save_path, persons):
    '''
    Load embeddings from files
    '''
    embeddings = []
    true_labels = []
    for person in persons:
        file_path = os.path.join(save_path, f"testX_{person.lower()}.pkl")
        if os.path.exists(file_path):
            with open(file_path, 'rb') as file:
                data = pickle.load(file)
                embeddings.extend(data)
                true_labels.extend([person] * len(data))
    return np.array(embeddings), true_labels

##### Loading the model and feature vectors of the sunglasses set

In [ ]:
X, true_labels = load_embeddings(SAVE_MAIN_PATH, valid_labels)
model = load_model(MODEL_PATH)

##### Confusion matrix

In [ ]:
predicted_probs = model.predict(X)
predicted_labels = [valid_labels[np.argmax(prob)] if max(prob) > 0.5 else 'Unknown' for prob in predicted_probs]

cm = confusion_matrix(true_labels, predicted_labels, labels=valid_labels)
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

plt.figure(figsize=(10, 8))
sns.heatmap(cm_percent, annot=True, fmt=".1f", cmap="Blues", xticklabels=valid_labels, yticklabels=valid_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix with Percentage Values")
plt.show()

##### Testing the model's performance on single photos

In [ ]:
embedder = FaceNet()

def get_embeddings(face_img_path):
    '''
    Get embeddings from face image
    '''
    face_img = cv2.imread(face_img_path)
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img, axis=0)
    yhat = embedder.embeddings(face_img)
    return yhat[0]

In [ ]:
detector = MTCNN()

def detect_face(img_path):
  '''
  Detect face on image and return face image
  '''
  image = cv2.imread(img_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  faces = detector.detect_faces(image)

  if len(faces) == 0:
    print("No faces for: ", img_path)
    return None

  x1, y1, width, height = faces[0]['box']
  x1, y1 = abs(x1), abs(y1)
  x2, y2 = x1 + width, y1 + height

  face_image = image[y1:y2, x1:x2]

  cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
  plt.imshow(image)
  plt.axis('off')
  plt.show()
  return face_image

In [ ]:
valid_labels = ['Emilia', 'Joanna', 'Kacper', 'Karolina', 'Mirek', 'Sylwia', 'Łukasz', 'Mariusz', 'Ania']

def recognize_person(img_path, confidence_threshold=0.50):
  '''
  Recognize person on the image
  '''
  detected_face = detect_face(img_path)
  detected_face_bgr = cv2.cvtColor(detected_face, cv2.COLOR_RGB2BGR)
  cv2_imshow(detected_face_bgr)

  if detected_face is not None:
      cv2.imwrite("/content/result.jpg", detected_face)

      emb = get_embeddings("/content/result.jpg")
      emb = np.expand_dims(emb, axis=0)
      prediction = model.predict(emb)

      max_prob = np.max(prediction)
      predicted_class_index = np.argmax(prediction)

      if max_prob >= confidence_threshold:
          predicted_class = valid_labels[predicted_class_index]
          print(f'Predicted Class: {predicted_class}')
          print(f'Max Probability: {max_prob}')
      else:
          print(f'Not confident enough. Predicted Class: Unknown')
          print(f'Max Probability: {max_prob}')

In [ ]:
for img in os.listdir('/content/drive/MyDrive/TESTED_IMAGES/Karolina'):
  recognize_person(os.path.join('/content/drive/MyDrive/TESTED_IMAGES/Karolina', img))

In [ ]:
for img in os.listdir('/content/drive/MyDrive/TESTED_IMAGES/Kacper'):
  recognize_person(os.path.join('/content/drive/MyDrive/TESTED_IMAGES/Kacper', img))

In [ ]:
for img in os.listdir('/content/drive/MyDrive/TESTED_IMAGES/Sylwia'):
  recognize_person(os.path.join('/content/drive/MyDrive/TESTED_IMAGES/Sylwia', img))

In [ ]:
for img in os.listdir('/content/drive/MyDrive/TESTED_IMAGES/Łukasz'):
  recognize_person(os.path.join('/content/drive/MyDrive/TESTED_IMAGES/Łukasz', img))

In [ ]:
for img in os.listdir('/content/drive/MyDrive/TESTED_IMAGES/Joanna'):
  recognize_person(os.path.join('/content/drive/MyDrive/TESTED_IMAGES/Joanna', img))

In [ ]:
for img in os.listdir('/content/drive/MyDrive/TESTED_IMAGES/Mirek'):
  recognize_person(os.path.join('/content/drive/MyDrive/TESTED_IMAGES/Mirek', img))

In [ ]:
for img in os.listdir('/content/drive/MyDrive/TESTED_IMAGES/Emilia'):
  recognize_person(os.path.join('/content/drive/MyDrive/TESTED_IMAGES/Emilia', img))

In [ ]:
for img in os.listdir('/content/drive/MyDrive/TESTED_IMAGES/Mariusz'):
  recognize_person(os.path.join('/content/drive/MyDrive/TESTED_IMAGES/Mariusz', img))

In [ ]:
for img in os.listdir('/content/drive/MyDrive/TESTED_IMAGES/Ania'):
  recognize_person(os.path.join('/content/drive/MyDrive/TESTED_IMAGES/Ania', img))